In [1]:
from data_tokenizing import tokenize_data
from data_nlp import show_nlp_results
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
import time
import numpy as np
from sklearn.pipeline import make_pipeline, Pipeline

2023-05-22 21:27:24.931082: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(X_train, X_test, y_train, y_test, embedding_matrix, max_len) = tokenize_data()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(len(X_train[0]), len(X_test[0]))
print(X_train[0])
print(len(embedding_matrix))
print(max_len)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daniel.chancir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/daniel.chancir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/daniel.chancir/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/Users/daniel.chancir/Documents/GitHub/aprendizaje_maquinas/data_cleaning.py:45: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


(39665, 1000)
(39665,)
(9917, 1000)
(9917,)
1000 1000
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0   

In [3]:
X_train = np.array(X_train)
Y_train = np.array(y_train)
x_test = np.array(X_test)
y_test = np.array(y_test)

In [4]:
X_train, X_val = X_train[:-5000], X_train[-5000:]
y_train, y_val = y_train[:-5000], y_train[-5000:]

In [5]:
import pandas as pd
pd.DataFrame(X_train).max().max()

4999

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import ParameterGrid
from keras.callbacks import EarlyStopping

embedding_dim = 100

def create_model(filters = 64, kernel_size = 3, strides=1, units = 256, 
                 optimizer='adam', rate = 0.25):
    model = Sequential()
    # Embedding layer
    model.add(Embedding(len(embedding_matrix) , 100, weights=[embedding_matrix], input_length= max_len))
    # Convolutional Layer(s)
    model.add(Conv1D(filters = filters, kernel_size = kernel_size, strides= strides, 
                     padding='valid', activation= 'relu'))
    model.add(MaxPooling1D(pool_size = 7))
    model.add(GlobalMaxPooling1D())
    # Dense layer(s)
    model.add(Dense(units = units, activation= 'relu'))
    model.add(Dropout(rate))
    # Output layer
    model.add(Dense(1, activation= 'sigmoid'))
    
    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer= optimizer,
                  metrics=['accuracy'])
    return model
# Build the model
model = KerasClassifier(build_fn= create_model)


/var/folders/x8/21809vns62d21gjxm06y07080000gp/T/ipykernel_21233/321455516.py:31: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn= create_model)


In [9]:
# Set the hyperparameters
filters = [128] #[64, 128, 256]
kernel_size = [3] #[3, 5, 7]
strides= [1] # [1, 2, 5]
Dense_units = [32, 128, 512]
rate_dropouts = [0.25] #[0.1, 0.25, 0.5]
optimizers = ['adam'] #['adam','rmsprop']
epochs = [5]
batches = [64] #[32, 64, 128]
# ----------------------------------------------
# Exhaustive Grid Search
param_grid = dict(optimizer= optimizers, epochs= epochs, batch_size= batches,
                  filters = filters, kernel_size = kernel_size, strides = strides, 
                  units = Dense_units, rate = rate_dropouts)

grid = ParameterGrid(param_grid)
param_sets = list(grid)

param_scores = []
for params in grid:

    print(params)
    model.set_params(**params)

    earlystopper = EarlyStopping(monitor='val_accuracy', patience= 0, verbose=1)
    
    history = model.fit(X_train, y_train,
                        shuffle= True,
                        validation_data=(X_val, y_val),
                        callbacks= [earlystopper])

    param_score = history.history['val_accuracy']
    param_scores.append(param_score[-1])
    print('+-'*50) 

p = np.argmax(np.array(param_scores))
print('param_scores:', param_scores)
print("best score:", param_scores[p])
# Choose best parameters
best_params = param_sets[p]
print("best parameter set", best_params)

{'batch_size': 64, 'epochs': 5, 'filters': 128, 'kernel_size': 3, 'optimizer': 'adam', 'rate': 0.25, 'strides': 1, 'units': 32}
Epoch 1/5
542/542 [==============================] - 87s 159ms/step - loss: 0.5240 - accuracy: 0.7316 - val_loss: 0.3785 - val_accuracy: 0.8354
Epoch 2/5
542/542 [==============================] - 86s 159ms/step - loss: 0.3287 - accuracy: 0.8638 - val_loss: 0.3206 - val_accuracy: 0.8604
Epoch 3/5
542/542 [==============================] - 86s 158ms/step - loss: 0.2369 - accuracy: 0.9083 - val_loss: 0.3260 - val_accuracy: 0.8648
Epoch 4/5
542/542 [==============================] - 87s 160ms/step - loss: 0.1635 - accuracy: 0.9432 - val_loss: 0.3431 - val_accuracy: 0.8628
Epoch 4: early stopping
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
{'batch_size': 64, 'epochs': 5, 'filters': 128, 'kernel_size': 3, 'optimizer': 'adam', 'rate': 0.25, 'strides': 1, 'units': 128}
Epoch 1/5
542/542 [=======================

In [10]:
model.set_params(**best_params)
model.fit(X_train, y_train)

Epoch 1/5
542/542 [==============================] - 84s 154ms/step - loss: 0.5297 - accuracy: 0.7269
Epoch 2/5
542/542 [==============================] - 83s 154ms/step - loss: 0.3306 - accuracy: 0.8621
Epoch 3/5
542/542 [==============================] - 381s 704ms/step - loss: 0.2388 - accuracy: 0.9087
Epoch 4/5
542/542 [==============================] - 83s 154ms/step - loss: 0.1627 - accuracy: 0.9420
Epoch 5/5
542/542 [==============================] - 157s 290ms/step - loss: 0.1006 - accuracy: 0.9683


In [11]:
print("Test accuracy = %f%%" % (accuracy_score(y_test, model.predict(X_test))*100))

310/310 [==============================] - 5s 15ms/step
Test accuracy = 85.479480%
